In [1]:
import json, csv

In [8]:
scl = '''

// Purities
#MB_Hold_Reg_Real[11] := "Purity Meter_DB"."Purity Downstream";
#MB_Hold_Reg_Real[12] := "Purity Meter_DB"."Purity Upstream";
#MB_Hold_Reg_Real[13] := "AIT501".AI.Value;

// Analog Devices
#MB_Hold_Reg_Real[14] := "DPT511".AI.Value;
#MB_Hold_Reg_Real[15] := "LT524".AI.Value2;
#MB_Hold_Reg_Real[16] := "PT305".AI.Value;
#MB_Hold_Reg_Real[17] := "PT306".AI.Value;
#MB_Hold_Reg_Real[18] := "PT503".AI.Value;
#MB_Hold_Reg_Real[19] := "PT520".AI.Value;
#MB_Hold_Reg_Real[20] := "TE521".AI.Value;
#MB_Hold_Reg_Real[21] := "TE522".AI.Value;
#MB_Hold_Reg_Real[22] := "Alicat".AI.Value;

'''

In [3]:
'''Use for sending CSV to customer'''

file_name = 'P654_tag_map'
tag_map = []

for line in iter(scl.splitlines()):
    # Skip line if it is empty or is a comment
    if line.strip(' ') == '':
        continue
    if line[0:2] == '//':
        continue
    
    # Get register portion and tag portion
    reg, tag = line.split(" := ")

    # Get the bit number if it exists
    if '.%X' in reg:
        bit = reg[-1]
    else:
        bit = None

    # Get the register index
    idx = int(reg[reg.find('[')+1:reg.rfind(']')])-1

    # Remove any comment at the end of the line
    tag = tag.split('//')[0]
    #Remove the end of line semicolon
    tag = tag.strip(';')
    # Get the scaling value 
    if '*' in tag:
        tag, scale = tag.split(' * ')
    else:
        scale = 1
    
    # Save relevant information to the output tag map
    temp = {}
    temp['index'] = idx
    temp['bit'] = bit
    temp['tag'] = tag
    temp['scale'] = int(scale) 
    tag_map.append(temp)

with open(file_name + '.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['index', 'tag', 'bit', 'scale'])
    writer.writeheader()
    writer.writerows(tag_map)

print(tag_map)

[{'index': 0, 'bit': None, 'tag': '0', 'scale': 1}, {'index': 1, 'bit': None, 'tag': '"CB Liquefier_DB".State', 'scale': 1}, {'index': 2, 'bit': None, 'tag': '"Helium Recovery_DB".State', 'scale': 1}, {'index': 3, 'bit': None, 'tag': '"System Status".AlarmState', 'scale': 1}, {'index': 4, 'bit': None, 'tag': '0', 'scale': 1}, {'index': 5, 'bit': None, 'tag': '"CB Liquefier_DB".Alarm', 'scale': 1}, {'index': 6, 'bit': None, 'tag': '"Helium Recovery_DB".Alarm', 'scale': 1}, {'index': 7, 'bit': None, 'tag': '"Purity Meter_DB".Alarm', 'scale': 1}, {'index': 8, 'bit': None, 'tag': '"CB Liquefier_DB".Trip', 'scale': 1}, {'index': 9, 'bit': None, 'tag': '"Helium Recovery_DB".Trip', 'scale': 1}, {'index': 10, 'bit': None, 'tag': '"HP Compressor_DB".State', 'scale': 1}, {'index': 11, 'bit': None, 'tag': '"HP Compressor_DB".Alarm', 'scale': 1}, {'index': 12, 'bit': None, 'tag': '"HP Compressor_DB".Trip', 'scale': 1}, {'index': 13, 'bit': None, 'tag': '"DPT508".AI.Value', 'scale': 100}, {'index':

In [ ]:
'''Integers for StrideLinx'''

# Added to the file name
project_number = '707'

# Lists for the two tag tables that are populated in StrideLinx
# Variables: Used to access the register from the PLC
# Logging: Used to set the logging type and frequency of the variable
tag_map_variables = []
tag_map_logging = []

for line in iter(scl.splitlines()):
    # Skip line if it is empty or is a comment
    if line.strip(' ') == '':
        continue
    if line[0:2] == '//':
        continue
    
    # Get register portion and tag portion
    reg, tag = line.split(" := ")

    # Get the register index
    idx = int(reg[reg.find('[')+1:reg.rfind(']')])-1

    # Remove any comment at the end of the line
    tag = tag.split('//')[0]
    #Remove the end of line semicolon
    tag = tag.strip(';').replace('"', '')

    # Get the scaling value 
    if '*' in tag:
        tag, scale = tag.split(' * ')
    else:
        scale = 1
    
    # Check if the value is a bool conversion
    if 'BOOL_TO_INT' in tag:
        tag = tag[:-1]
        tag = tag.split('(')[1]
    
    # Save relevant information to the StrideLinx Variable tag map
    temp = {}
    temp['Identifier'] = ''
    temp['Name'] = tag
    temp['Address'] = idx
    temp['Type'] = 'int'
    temp['Width'] = 16
    temp['Signed'] = True
    temp['Max string length'] = ''
    temp['Factor'] = 1 / int(scale)
    temp['Unit'] = ''
    tag_map_variables.append(temp)

    # Save relevant information to the StrideLinx Logging tag map
    temp2 = {}
    temp2['Identifier'] = ''
    temp2['Address'] = idx
    temp2['Name'] = tag
    temp2['Logging interval'] = '10s'
    temp2['Retention policy'] = '364w'
    temp2['Edge aggregator'] = 'last'
    temp2['On change expiry'] = ''
    # Used to find tags that should be logged on change. Populate list with other signal names.
    if any(x in tag for x in ['XY', 'Run', 'Feedback', 'HE', 'Trip', 'Alarm', 'AlarmState', 'State', 'TransitionPurityMeter', 'MV', 'P501']):
        temp2['Log event'] = 'change'
    else:
        temp2['Log event'] = 'interval'
    temp2['Trigger address'] = ''
    temp2['Trigger condition'] = ''
    temp2['Trigger threshold'] = ''
    tag_map_logging.append(temp2)

# Save the variables to a csv that can be uploaded to the StrideLinx Fleet Manager
with open(project_number + '_StrideLinx_Int_Variables.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['Identifier', 'Name', 'Address', 'Type', 'Width', 'Signed', 'Max string length', 'Factor', 'Unit'])
    writer.writeheader()
    writer.writerows(tag_map_variables)

# Save the logging tags to a csv that can be uploaded to the StrideLinx Fleet Manager
with open(project_number + '_StrideLinx_Int_Logging.csv', 'w', newline='') as f:
    # ['Identifier', 'Address', 'Name', 'Logging interval', 'Retention policy', 'Edge aggregator',	'On change expiry',	'Log event', 'Trigger address',	'Trigger condition', 'Trigger threshold'
    writer = csv.DictWriter(f, fieldnames=['Identifier', 'Address', 'Name', 'Logging interval', 'Retention policy', 'Edge aggregator',	'On change expiry',	'Log event', 'Trigger address',	'Trigger condition', 'Trigger threshold'])
    writer.writeheader()
    writer.writerows(tag_map_logging)

print(tag_map_variables)

[{'Identifier': '', 'Name': 'CH Liquefier_DB.State', 'Address': 1, 'Type': 'int', 'Width': 16, 'Signed': True, 'Max string length': '', 'Factor': 1.0, 'Unit': ''}, {'Identifier': '', 'Name': 'Helium Recovery_DB.State', 'Address': 2, 'Type': 'int', 'Width': 16, 'Signed': True, 'Max string length': '', 'Factor': 1.0, 'Unit': ''}, {'Identifier': '', 'Name': 'System Status.AlarmState', 'Address': 3, 'Type': 'int', 'Width': 16, 'Signed': True, 'Max string length': '', 'Factor': 1.0, 'Unit': ''}, {'Identifier': '', 'Name': 'CH Liquefier_DB.Alarm', 'Address': 5, 'Type': 'int', 'Width': 16, 'Signed': True, 'Max string length': '', 'Factor': 1.0, 'Unit': ''}, {'Identifier': '', 'Name': 'Helium Recovery_DB.Alarm', 'Address': 6, 'Type': 'int', 'Width': 16, 'Signed': True, 'Max string length': '', 'Factor': 1.0, 'Unit': ''}, {'Identifier': '', 'Name': 'Purity Meter_DB.Alarm', 'Address': 7, 'Type': 'int', 'Width': 16, 'Signed': True, 'Max string length': '', 'Factor': 1.0, 'Unit': ''}, {'Identifier

In [10]:
'''Reals for StrideLinx'''

# Added to the file name
project_number = '707'

# Lists for the two tag tables that are populated in StrideLinx
# Variables: Used to access the register from the PLC
# Logging: Used to set the logging type and frequency of the variable
tag_map_variables = []
tag_map_logging = []

for line in iter(scl.splitlines()):
    # Skip line if it is empty or is a comment
    if line.strip(' ') == '':
        continue
    if line[0:2] == '//':
        continue
    
    # Get register portion and tag portion
    reg, tag = line.split(" := ")

    # Get the register index
    idx = (int(reg[reg.find('[')+1:reg.rfind(']')])-1) * 2

    # Remove any comment at the end of the line
    tag = tag.split('//')[0]
    #Remove the end of line semicolon
    tag = tag.strip(';').replace('"', '')
    
    # Save relevant information to the StrideLinx Variable tag map
    temp = {}
    temp['Identifier'] = ''
    temp['Name'] = tag
    temp['Address'] = idx
    temp['Type'] = 'float'
    temp['Width'] = 32
    temp['Signed'] = ''
    temp['Max string length'] = ''
    temp['Factor'] = ''
    temp['Unit'] = ''
    tag_map_variables.append(temp)

    # Save relevant information to the StrideLinx Logging tag map
    temp2 = {}
    temp2['Identifier'] = ''
    temp2['Address'] = idx
    temp2['Name'] = tag
    temp2['Logging interval'] = '10s'
    temp2['Retention policy'] = '364w'
    temp2['Edge aggregator'] = 'last'
    temp2['On change expiry'] = ''
    # Used to find tags that should be logged on change. Populate list with other signal names.
    if any(x in tag for x in ['XY', 'Run', 'Feedback', 'HE', 'Trip', 'Alarm', 'AlarmState', 'State', 'TransitionPurityMeter', 'MV', 'P501']):
        temp2['Log event'] = 'change'
    else:
        temp2['Log event'] = 'interval'
    temp2['Trigger address'] = ''
    temp2['Trigger condition'] = ''
    temp2['Trigger threshold'] = ''
    tag_map_logging.append(temp2)

# Save the variables to a csv that can be uploaded to the StrideLinx Fleet Manager
with open(project_number + '_StrideLinx_Real_Variables.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['Identifier', 'Name', 'Address', 'Type', 'Width', 'Signed', 'Max string length', 'Factor', 'Unit'])
    writer.writeheader()
    writer.writerows(tag_map_variables)

# Save the logging tags to a csv that can be uploaded to the StrideLinx Fleet Manager
with open(project_number + '_StrideLinx_Real_Logging.csv', 'w', newline='') as f:
    # ['Identifier', 'Address', 'Name', 'Logging interval', 'Retention policy', 'Edge aggregator',	'On change expiry',	'Log event', 'Trigger address',	'Trigger condition', 'Trigger threshold'
    writer = csv.DictWriter(f, fieldnames=['Identifier', 'Address', 'Name', 'Logging interval', 'Retention policy', 'Edge aggregator',	'On change expiry',	'Log event', 'Trigger address',	'Trigger condition', 'Trigger threshold'])
    writer.writeheader()
    writer.writerows(tag_map_logging)

print(tag_map_variables)

[{'Identifier': '', 'Name': 'Purity Meter_DB.Purity Downstream', 'Address': 20, 'Type': 'float', 'Width': 32, 'Signed': '', 'Max string length': '', 'Factor': '', 'Unit': ''}, {'Identifier': '', 'Name': 'Purity Meter_DB.Purity Upstream', 'Address': 22, 'Type': 'float', 'Width': 32, 'Signed': '', 'Max string length': '', 'Factor': '', 'Unit': ''}, {'Identifier': '', 'Name': 'AIT501.AI.Value', 'Address': 24, 'Type': 'float', 'Width': 32, 'Signed': '', 'Max string length': '', 'Factor': '', 'Unit': ''}, {'Identifier': '', 'Name': 'DPT511.AI.Value', 'Address': 26, 'Type': 'float', 'Width': 32, 'Signed': '', 'Max string length': '', 'Factor': '', 'Unit': ''}, {'Identifier': '', 'Name': 'LT524.AI.Value2', 'Address': 28, 'Type': 'float', 'Width': 32, 'Signed': '', 'Max string length': '', 'Factor': '', 'Unit': ''}, {'Identifier': '', 'Name': 'PT305.AI.Value', 'Address': 30, 'Type': 'float', 'Width': 32, 'Signed': '', 'Max string length': '', 'Factor': '', 'Unit': ''}, {'Identifier': '', 'Name

In [5]:
'''JSON Export'''
# Serializing json
json_object = json.dumps(tag_map, indent=4)
 
# Writing to sample.json
with open("663_tag_map.json", "w") as outfile:
    outfile.write(json_object)

In [9]:
'''CSV Export'''
with open('654_tag_map.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['index', 'tag', 'bit', 'scale'])
    writer.writeheader()
    writer.writerows(tag_map)